In [2]:
import os
import asyncio
import logging 
from pathlib import Path
from typing import Tuple, Any, List, Dict, Optional
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_cohere import CohereEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import (
    TextLoader, 
    UnstructuredMarkdownLoader,
    JSONLoader,
    UnstructuredHTMLLoader,
    PyPDFLoader
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain.callbacks.base import AsyncCallbackHandler
from langchain.schema import LLMResult

load_dotenv()
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
@dataclass
class SelfRAGResponse:
    """Complete self rag with reflection"""
    answer: str
    retrieved_docs: List[Document]
    reflection_score: float
    needs_retrieval: bool
    citations: List[str]
    retrieval_decision_reasoning: str


class RateLimitCallback(AsyncCallbackHandler):
    """Callback handler to manage API rate limiting with semaphores"""
    
    def __init__(self, semaphore: asyncio.Semaphore):
        self.semaphore = semaphore
        
    async def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> None:
        await self.semaphore.acquire()
        
    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        self.semaphore.release()

In [ ]:
class DocumentLoader:
    def __init__(self):
        self.loaders = {
            '.txt': TextLoader,
            '.md': UnstructuredMarkdownLoader,
            '.json': self._create_json_loader,
            '.html': UnstructuredHTMLLoader,
            '.py': TextLoader,
            '.js': TextLoader,
            '.css': TextLoader,
            '.pdf': PyPDFLoader
        }

    def _create_json_loader(self, file_path: str):
        """Create JSON loader with custom jq_schema"""
        return JSONLoader(
            file_path=file_path,
            jq_schema='.[]',
            text_content=False
        )

    async def load_documents(self, kb_folder: str) -> List[Document]:
        """Load all documents from the knowledge base folder"""
        documents = []
        kb_path = Path(kb_folder)

        if not kb_path.exists():
            raise FileNotFoundError(f"Knowledge base folder not found: {kb_path}")

        for file_path in kb_path.glob("**/*"):
            if file_path.is_file() and file_path.suffix.is_lower() in self.loaders:
                try:
                    loader_class = self.loaders[file_path.suffix.is_lower()]

                    if file_path.suffix.lower() == ".json":
                        loader = loader_class(str(file_path))
                    else:
                        loader = loader_class(str(file_path))

                    docs = loader.load()

                    # Add metadata
                    for doc in docs:
                        doc.metadata.update({
                            'file_path': str(file_path),
                            'file_type': file_path.suffix,
                            'file_name': file_path.name
                        })

                    documents.extend(docs)
                
                except Exception as e:
                    logger.warning(f"There was an error loading the knowledge base: {str(e)}")
                    # Fallback to TextLoader for unknown formats
                    try:
                        loader = TextLoader(str(file_path))
                        docs = loader.load()
                        # Add metadata
                        for doc in docs:
                            doc.metadata.update({
                                'file_path': str(file_path),
                                'file_type': file_path.suffix,
                                'file_name': file_path.name
                            })

                        documents.extend(docs)

                    except Exception as fallback_error:
                        logger.error(f"Failed to load {file_path} with fallback: {fallback_error}")
        
        logger.info(f"Loaded {len(documents)} documents from {kb_folder}")
        return documents

_IncompleteInputError: incomplete input (3048260000.py, line 42)

In [5]:
loaders = {
            '.txt': TextLoader,
            '.md': UnstructuredMarkdownLoader,
            #'.json': self._create_json_loader,
            '.html': UnstructuredHTMLLoader,
            '.py': TextLoader,
            '.js': TextLoader,
            '.css': TextLoader,
            '.pdf': PyPDFLoader
        }

loaders[".txt"](str("books"))